# 크롤링 시작하기

## 함수 시작

In [3]:
class MovieAPI():
    def __init__(self):
        self.index = []
        self.title_li = []
        self.link_li = []
        self.image_li = []
        self.year_li = []
        self.dir_li = []
        self.actor_li = []
        self.rating_li = []
        self.error_index = []
    
    def get_movie(self, index_list, movie_list, dir_list, year_list, client_id, client_secret):
        import urllib.request
        import xmltodict
        import pandas as pd
        import time # 중간중간 멈춤
        
        for i, movie in enumerate(movie_list):
            encText = urllib.parse.quote(movie)
            url = 'https://openapi.naver.com/v1/search/movie.xml?query="' + encText  + f'"&display=100&yearfrom={str(int(year_list[i])-1)}&yearto={str(int(year_list[i]))}'
            time.sleep(0.5)
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            if(rescode==200): #잘 긇어와졌으면
                response_body = response.read()
                dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False) #불러온 파일을 딕셔너리로 저장
                search_li = dict_data['rss']['channel']
                if 'item' in search_li.keys():
                    search_li = search_li['item']
                    if type(search_li) == list:
                        for j in range(len(search_li)):
                            mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
                            mv_rating = search_li[j]['userRating']
                            mv_dir = search_li[j]['director']
                            mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                            mv_actor = search_li[j]['actor']
                            mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                            mv_year = search_li[j]['pubDate']
                            if mv_title.replace(' ','') == movie.replace(' ','') and mv_rating != None and mv_dir != None and mv_actor != None: # 영화 제목의 공백을 전부 지우고 비교하고자하는 영화 제목 리스트와 비교
                                if mv_rating != '0.00' and dir_list[i] in mv_dir:
                                    self.index.append(index_list[i])
                                    self.title_li.append(mv_title)
                                    self.link_li.append(search_li[j]['link'])
                                    self.image_li.append(search_li[j]['image'])
                                    self.year_li.append(mv_year)
                                    self.dir_li.append(mv_dir)
                                    self.actor_li.append(mv_actor)
                                    self.rating_li.append(mv_rating)
                                    break
                                elif self.index != [] and self.index[-1] != i and mv_rating != '0.00' and dir_list[i] not in mv_dir: # 감독이 일치하지 않을 때
                                    self.index.append(index_list[i])
                                    self.title_li.append(mv_title)
                                    self.link_li.append(search_li[j]['link'])
                                    self.image_li.append(search_li[j]['image'])
                                    self.year_li.append(search_li[j]['pubDate'])
                                    self.dir_li.append(mv_dir)
                                    self.actor_li.append(mv_actor)
                                    self.rating_li.append(mv_rating)
                                else:
                                    self.error_index.append([index_list[i], '2nd'])
                                    break
                            else:
                                self.error_index.append([index_list[i], '1st'])
                                continue
                    elif type(search_li) == dict:
                        mv_title = search_li['title'].replace('</b>','').replace('<b>','')
                        mv_dir = search_li['director']
                        mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                        mv_actor = search_li['actor']
                        mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

                        self.index.append(index_list[i])
                        self.title_li.append(mv_title)
                        self.link_li.append(search_li['link'])
                        self.image_li.append(search_li['image'])
                        self.year_li.append(search_li['pubDate'])
                        self.dir_li.append(mv_dir)
                        self.actor_li.append(mv_actor)
                        self.rating_li.append(search_li['userRating'])
                    else:
                        self.error_index.append([index_list[i], 'list/dict 분류 오류'])
            else:
                print("Error Code:" + rescode)
            if (i != 0 and i % 10 == 0) or i == (len(movie_list)-1):
                print(f'{index_list[i]}번째 완료')
            if i != 0 and i % 500 == 0:    # 500번마다 savepoint로 저장해두기 
                pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose().to_csv(f'./files/savepoint{index_list[i]}_df_utf-8.csv', encoding='utf-8')
                print('savepoint', index_list[i]) #transpose 행과 열 위치 변경
                
        movie_df = pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose()
        movie_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '연도', '감독', '출연', '평점']
        movie_df = movie_df.set_index('순번')

        return movie_df

## 리스트 불러오기

In [9]:
import pandas as pd

# 리스트 불러오기
movie_df = pd.read_csv('movie_resize2.csv', encoding='utf-8')
display(movie_df[:6])
index_list = list(movie_df.index)
movie_list = movie_df['영화명'].to_list()
dir_list = movie_df['감독'].to_list()
year_list = movie_df['연도'].to_list()

print('len =',len(index_list,), index_list[:10])
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])
print('len =',len(year_list), year_list[:10])


,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분
0,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,사극,15세관람가,일반영화
1,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,코미디,15세관람가,일반영화
2,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,판타지,12세관람가,일반영화
3,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,드라마,12세관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,액션,12세관람가,일반영화
5,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21,장편,미국,2648,114810421450,13747792,애니메이션,전체관람가,일반영화


len = 4145 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len = 4145 ['명량', '극한직업', '신과함께-죄와 벌', '국제시장', '어벤져스: 엔드게임', '겨울왕국 2', '아바타', '베테랑', '도둑들', '7번방의 선물']
len = 4145 ['김한민', '이병헌', '김용화', '윤제균', '안소니 루소', '제니퍼 리', '제임스 카메론', '류승완', '최동훈', '이환경']
len = 68 [2017, 2013, 2009, 2013, 2011, 2007, 2008, 2009, 2008, 2011]


## 함수 실행하기

In [ ]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

# 함수 실행
ma = MovieAPI()
naver_df = ma.get_movie(movie_list=movie_list, dir_list=dir_list, client_id=client_id, client_secret=client_secret)

# 결과 데이터 중복 제거
df_drop_dp = naver_df.drop_duplicates(['영화명'], keep='last')
df_drop_dp

# 결과 데이터와 중복 제거 데이터 저장
naver_df.to_csv('./files/naver_api_utf-8.csv', encoding='utf-8')
df_drop_dp.to_csv('./files/naver_api_drop_dupl_utf-8.csv', encoding='utf-8')
# display(naver_df)


# ▼ 아래는 함수 오류 확인용

In [72]:
naver_df = ma.get_movie(movie_list=movie_list[1282:1285], dir_list=dir_list, client_id=client_id, client_secret=client_secret)

2번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [129]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=['어벤져스: 인피니티 워'], dir_list=['안소니 루소'], client_id=client_id, client_secret=client_secret)

# naver_df.to_csv('naver_api_cp949_0.csv', encoding='cp949')
display(naver_df)

0번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
0,어벤져스: 인피니티 워,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/13...,2018,"[앤서니 루소, 조 루소]","[크리스 프랫, 조 샐다나, 브래들리 쿠퍼, 빈 디젤, 조슈 브롤린, 엘리자베스 올...",8.97


## 1개씩 테스트

In [286]:
import urllib.request
import xmltodict
import pandas as pd
import time
#나잇&데이	제임스 맨골드	2010
index = []
year_list = ['2010']
dir_list=['제임스 맨골드']

for i, movie in enumerate(['나잇&데이']):
    encText = urllib.parse.quote(movie)
    url = 'https://openapi.naver.com/v1/search/movie.xml?query="' + encText  + f'"&display=100&yearfrom={str(int(year_list[i])-1)}&yearto={str(int(year_list[i]))}'
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
        search_li = dict_data['rss']['channel']
        if 'item' in search_li.keys():
            search_li = search_li['item']
            print(search_li)
            for j in range(len(search_li)):
                mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
                mv_rating = search_li[j]['userRating']
                mv_dir = search_li[j]['director']
                mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                mv_actor = search_li[j]['actor']
                mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                print('if 1:', mv_title.replace(' ','') == movie.replace(' ','') and mv_rating != None and mv_dir != None and mv_actor != None, dir_list[i], mv_dir)
                print(mv_title, movie, dir_list[i], mv_dir)
                if mv_title.replace(' ','') == movie.replace(' ','') and mv_rating != None and mv_dir != None and mv_actor != None:
                    print('if 2:', mv_rating != '0.00' and dir_list[i] in mv_dir)
                    if mv_rating != '0.00' and dir_list[i] in mv_dir:
                        print(mv_title,search_li[j]['link'],search_li[j]['image'],search_li[j]['pubDate'],mv_dir,mv_actor,mv_rating)
                    elif index != [] and index[-1] != i and mv_rating != '0.00' and dir_list[i] not in mv_dir:
                        print(mv_title,search_li[j]['link'],search_li[j]['image'],search_li[j]['pubDate'],mv_dir,mv_actor,mv_rating)
                    else: print(0)
                else: print(0)


{'title': '<b>나잇 &amp; 데이</b>', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=71486', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0714/71486_P32_121349.jpg', 'subtitle': 'Knight &amp; Day', 'pubDate': '2010', 'director': '제임스 맨골드|', 'actor': '톰 크루즈|카메론 디아즈|', 'userRating': '8.38'}


KeyError: 0

In [259]:
print(url)

https://openapi.naver.com/v1/search/movie.xml?query="%EB%A7%88%EB%8D%94"&display=100&yearfrom=2007&yearto=2011


In [275]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=['월드 워 Z', '마더', '친구 2'], dir_list=['샘 햄슨', '봉준호', '곽경택'], year_list=['2013', '2009', '2013'], client_id=client_id, client_secret=client_secret)

# naver_df.to_csv('naver_api_utf-8_missed2.csv', encoding='utf-8')
display(naver_df)

year_list=['2013', '2009', '2013']
df_dropped = naver_df.drop_duplicates(['영화명'], keep='last') # 마지막에 영화문(브레이크에 의해 찾아진)를 선책하기
df_dropped

2번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
0,월드워Z,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/05...,2013,[마크 포스터],"[브래드 피트, 미레유 에노스]",8.24
1,마더,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/07...,2010,"[미즈타 노부오, 나가누마 마코토]","[마츠유키 야스코, 사카이 와카나]",9.64
1,마더,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,2009,[봉준호],"[김혜자, 원빈]",8.19
2,친구2,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,2013,[곽경택],"[유오성, 주진모, 김우빈]",6.20


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
0,월드워Z,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/05...,2013,[마크 포스터],"[브래드 피트, 미레유 에노스]",8.24
1,마더,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,2009,[봉준호],"[김혜자, 원빈]",8.19
2,친구2,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,2013,[곽경택],"[유오성, 주진모, 김우빈]",6.20


In [149]:
import numpy as np

movie_df = pd.read_csv('df_years_utf-8.csv', encoding='utf-8')[['영화명','감독','연도']]
missed_df = pd.read_csv('missed_cp949.csv', encoding='cp949', index_col=0)

display(movie_df.index.to_numpy(), missed_df.index.to_numpy())

missed_index = np.intersect1d(movie_df.index.to_numpy(), missed_df.index.to_numpy())
print(missed_index)


array([   0,    1,    2, ..., 4142, 4143, 4144], dtype=int64)

array([   6,    8,   10, ..., 4140, 4143, 4144], dtype=int64)

[   6    8   10 ... 4140 4143 4144]


In [151]:
import pandas as pd

missed_year_df = movie_df.loc[missed_index]

movie_list = missed_year_df['영화명'].to_list()
dir_list = missed_year_df['감독'].to_list()
year_list = missed_year_df['연도'].to_list()
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])
print('len =',len(dir_list), year_list[:10])

client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=movie_list, dir_list=dir_list, year_list=year_list, client_id=client_id, client_secret=client_secret)

naver_df.to_csv('naver_api_utf-8_missed2.csv', encoding='utf-8')
display(len(naver_df))

len = 1059 ['아바타', '도둑들', '암살', '어벤져스: 인피니티 워', '겨울왕국', '백두산', '써니', '스파이더맨: 홈 커밍', '베를린', '마스터']
len = 1059 ['제임스 카메론', '최동훈', '최동훈', '안소니 루소', '제니퍼 리', '김병서', '강형철', '존 왓츠', '류승완', '조의석']
len = 1059 [2009, 2012, 2015, 2018, 2014, 2019, 2011, 2017, 2013, 2016]


In [161]:
naver_df.to_csv('naver_api_utf-8_missed2.csv', encoding='utf-8')

In [165]:
missed_index = missed_year_df.reset_index()
missed_index

,index,영화명,감독,연도
0,6,아바타,제임스 카메론,2009
1,8,도둑들,최동훈,2012
2,10,암살,최동훈,2015
3,19,어벤져스: 인피니티 워,안소니 루소,2018
4,23,겨울왕국,제니퍼 리,2014
...,...,...,...,...
1054,4136,섀도우 클라우드,기타,2021
1055,4137,엠티맨,기타,2020
1056,4140,앙상블,정형석,2020
1057,4143,죽을 때까지,기타,2021


In [166]:
diff = np.setdiff1d(missed_index.index.to_list(), naver_df.index.to_list())
diff

array([   7,   18,   31,   33,   47,   50,   58,   62,   63,   64,   70,
         75,   76,   78,   81,   82,   85,   91,   92,   93,   94,  102,
        104,  111,  122,  126,  130,  132,  143,  145,  156,  162,  163,
        165,  167,  176,  181,  199,  203,  204,  207,  210,  214,  217,
        224,  226,  229,  241,  247,  248,  249,  250,  253,  255,  257,
        262,  264,  272,  274,  275,  279,  281,  331,  339,  356,  361,
        373,  381,  402,  410,  418,  467,  482,  497,  498,  499,  518,
        532,  546,  553,  580,  582,  590,  596,  603,  614,  630,  640,
        642,  647,  652,  660,  702,  724,  736,  752,  777,  792,  793,
        795,  820,  828,  833,  850,  923,  925,  926,  943,  945,  947,
        970,  973,  982,  988,  990,  999, 1000, 1040, 1047, 1049])

In [182]:
missed_year_df

,영화명,감독,연도
6,아바타,제임스 카메론,2009
8,도둑들,최동훈,2012
10,암살,최동훈,2015
19,어벤져스: 인피니티 워,안소니 루소,2018
23,겨울왕국,제니퍼 리,2014
...,...,...,...
4136,섀도우 클라우드,기타,2021
4137,엠티맨,기타,2020
4140,앙상블,정형석,2020
4143,죽을 때까지,기타,2021


In [207]:
diff_df = [missed_index['index'][i] for i in diff]
print(len(diff_df), len(diff))
display(diff_df)

120 120


[49,
 96,
 212,
 218,
 307,
 333,
 384,
 441,
 458,
 467,
 499,
 518,
 529,
 564,
 616,
 617,
 620,
 698,
 700,
 714,
 722,
 786,
 790,
 840,
 957,
 971,
 996,
 1007,
 1122,
 1138,
 1208,
 1272,
 1282,
 1294,
 1315,
 1379,
 1397,
 1542,
 1556,
 1559,
 1565,
 1586,
 1605,
 1622,
 1660,
 1672,
 1688,
 1777,
 1811,
 1812,
 1818,
 1819,
 1842,
 1851,
 1863,
 1917,
 1923,
 1960,
 1979,
 1990,
 2008,
 2013,
 2129,
 2152,
 2198,
 2209,
 2238,
 2255,
 2297,
 2319,
 2347,
 2474,
 2515,
 2566,
 2567,
 2568,
 2616,
 2651,
 2691,
 2706,
 2783,
 2794,
 2820,
 2835,
 2846,
 2884,
 2932,
 2956,
 2958,
 2970,
 2983,
 3006,
 3136,
 3187,
 3215,
 3263,
 3329,
 3369,
 3370,
 3375,
 3448,
 3471,
 3488,
 3560,
 3744,
 3759,
 3761,
 3831,
 3836,
 3843,
 3911,
 3921,
 3944,
 3958,
 3964,
 3993,
 3996,
 4103,
 4117,
 4125]

In [ ]:
diff_df2 = missed_year_df.loc[diff_df]
display(diff_df2)

In [ ]:
import pandas as pd

movie_list = diff_df2['영화명'].to_list()
dir_list = diff_df2['감독'].to_list()
year_list = diff_df2['연도'].to_list()
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])
print('len =',len(dir_list), year_list[:10])

client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=movie_list, dir_list=dir_list, year_list=year_list, client_id=client_id, client_secret=client_secret)

naver_df.to_csv('naver_api_utf-8_missed4.csv', encoding='utf-8')
display(len(naver_df))

In [236]:
diff_df3 = missed_year_df.loc[diff_li2]
display(diff_df3)

,영화명,감독,연도
96,월드 워 Z,마크 포스터,2013
212,마더,봉준호,2009
218,친구 2,곽경택,2013
307,나잇&데이,제임스 맨골드,2010
333,마이 웨이,강제규,2011
...,...,...,...
3993,맛있는 녀석들,순 하이펑,2020
3996,킬 스위치,기타,2020
4103,키퍼스,기타,2021
4117,맘&대드,브라이언 테일러,2018


## 4번째 크롤링

In [ ]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=['월드 워 Z', '마더', '친구 2'], dir_list=['샘 햄슨', '봉준호', '곽경택'], year_list=['2013', '2009', '2013'], client_id=client_id, client_secret=client_secret)

# naver_df.to_csv('naver_api_utf-8_missed2.csv', encoding='utf-8')
display(naver_df)

year_list=['2013', '2009', '2013']
df_dropped = naver_df.drop_duplicates(['영화명'], keep='last')
df_dropped

In [1]:
import pandas as pd
diff_df2 = pd.read_csv('naver_api_utf-8_missed4_droped_dupl.csv', encoding='utf-8')

In [4]:
import pandas as pd

movie_list = diff_df2['영화명'].to_list()
dir_list = diff_df2['감독'].to_list()
year_list = diff_df2['연도'].to_list()
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])
print('len =',len(dir_list), year_list[:10])

client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=movie_list, dir_list=dir_list, year_list=year_list, client_id=client_id, client_secret=client_secret)

df_dropped = naver_df.drop_duplicates(['영화명'], keep='last')
df_dropped

naver_df.to_csv('naver_api_utf-8_missed4.csv', encoding='utf-8')
df_dropped.to_csv('naver_api_utf-8_missed4_droped_dupl.csv', encoding='utf-8')

len(df_dropped)
len(naver_df)

len = 68 ['스파이더맨: 홈커밍', '월드워Z', '마더', '친구2', '마이웨이', '무방비 도시', '원스 어폰 어 타임', '작전', '예스 맨', '신들의 전쟁']
len = 68 ["['존 왓츠']", "['마크 포스터']", "['봉준호']", "['곽경택']", "['강제규']", "['이상기']", "['정용기']", "['이호재']", "['페이튼 리드']", "['타셈 싱']"]
len = 68 [2017, 2013, 2009, 2013, 2011, 2007, 2008, 2009, 2008, 2011]
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
67번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


68

In [5]:
a = pd.DataFrame(ma.error_index).drop_duplicates([0])

In [6]:
# 어디서 에러 떴는지 확인
err_li = diff_df2.reset_index().loc[a[0]]
err_li

,index,순번,영화명,네이버URL,이미지URL,연도,감독,출연,평점
2,2,2,마더,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,2009,['봉준호'],"['김혜자', '원빈']",8.19
3,3,3,친구2,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,2013,['곽경택'],"['유오성', '주진모', '김우빈']",6.20
4,4,5,마이웨이,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/07...,2011,['강제규'],"['장동건', '오다기리 죠', '판빙빙']",7.20
7,7,9,작전,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/05...,2009,['이호재'],"['박용하', '김민정', '박희순', '김무열', '조덕현', '김준성']",8.63
9,9,11,신들의 전쟁,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/07...,2011,['타셈 싱'],"['헨리 카빌', '미키 루크', '프리다 핀토']",6.48
10,10,12,영화는 영화다,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,2008,['장훈'],"['소지섭', '강지환']",8.85
11,11,13,바람 바람 바람,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/14...,2017,['이병헌'],"['이성민', '신하균', '송지효', '이엘']",6.20
12,12,14,업,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/05...,2009,"['피트 닥터', '밥 피터슨']","['에드워드 애스너', '조던 나가이', '크리스토퍼 플러머', '밥 피터슨']",9.33
13,13,15,시크릿,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/05...,2009,['윤재구'],"['차승원', '송윤아', '류승룡']",7.57
22,22,25,레드,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/07...,2010,['로베르트 슈벤트케'],"['브루스 윌리스', '모건 프리먼', '존 말코비치', '메리 루이스 파커', '...",8.07


In [332]:
tmp = np.setdiff1d(diff_df2['영화명'].to_numpy(), err_li['영화명'].to_numpy())

In [ ]:
# https://jimmy-ai.tistory.com/102
# df.drop_duplicates()

## 맨 아래

In [2]:
%pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 21.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 17.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import urllib.request
import xmltodict
import pandas as pd
import time
#도둑들	최동훈	2012
index = []
year_list = ['2012']
dir_list=['최동훈']

client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

for i, movie in enumerate(['도둑들']):
    encText = urllib.parse.quote(movie)
    url = 'https://openapi.naver.com/v1/search/movie.xml?query="' + encText  + f'"&display=100&yearfrom={str(int(year_list[i])-1)}&yearto={str(int(year_list[i]))}'
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
        display(dict_data)

{'rss': {'channel': {'title': "Naver Open API - movie ::'&quot;도둑들&quot;'",
   'link': 'https://search.naver.com',
   'description': 'Naver Search Result',
   'lastBuildDate': 'Thu, 23 Mar 2023 18:39:16 +0900',
   'total': '15',
   'start': '1',
   'display': '15',
   'item': [{'title': '<b>도둑들</b>',
     'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=78726',
     'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0787/78726_P17_143306.jpg',
     'subtitle': 'The Thieves',
     'pubDate': '2012',
     'director': '최동훈|',
     'actor': '김윤석|김혜수|이정재|전지현|임달화|김해숙|오달수|김수현|이신제|증국상|',
     'userRating': '7.65'},
    {'title': '열쇠 <b>도둑</b>의 방법',
     'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=96236',
     'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0962/96236_P02_110255.jpg',
     'subtitle': 'KEY OF LIFE',
     'pubDate': '2012',
     'director': '우치다 켄지|',
     'actor': '히로스에 료코|사카이 마사토|카가와 테루유키|아라카와 요시요시|모리구치 요우코|',
     'userRating': '8.43'},
   